In [42]:
values, gates = {}, []
with open("input24.txt", "r") as file:
    line = file.readline().strip("\n").split(":")
    while line != [""]:
        values[line[0]] = line[1][-1]=="1"
        line = file.readline().strip("\n").split(":")
    line = file.readline().strip("\n").split()
    while line != []:
        gates.append((line[0], line[2], line[1], line[4]))
        line = file.readline().strip("\n").split()

In [39]:
while gates:
    current = gates.pop(0)
    if current[0] in values and current[1] in values:
        if current[2] == "AND":
            values[current[3]] = values[current[0]] and values[current[1]]
        elif current[2] == "OR":
            values[current[3]] = values[current[0]] or values[current[1]]
        elif current[2] == "XOR":
            values[current[3]] = values[current[0]] != values[current[1]]
    else: gates.append(current)

In [40]:
z_condensed = [values[k] for k in sorted(list(values.keys())) if k[0]=="z"]
sum(2**(n)*z_condensed[n] for n in range(len(z_condensed)))

46463754151024

Part II

The idea: put together (on paper in my case) a circuit to make the addition. It has specific roles at every bit, like the carry, the inputs, the outpus, etc. For each bit, check if path from inputs (and carry) leads correctly to outputs (and another carry). Miswires will show up as errors.

In [1]:
gates = []
with open("input24.txt", "r") as file:
    line = file.readline().strip("\n").split(":")
    while line != [""]:
        line = file.readline().strip("\n").split(":")
    line = file.readline().strip("\n").split()
    while line != []:
        gates.append((line[0], line[2], line[1], line[4]))
        line = file.readline().strip("\n").split()

In [6]:
def binarize(x:int, length=None):
    if length is None: length = int(math.log2(x)) + 1
    return [bool(((x)//2**j)%2) for j in range(length)]

def run_gates(gates: list, x:int, y:int, xlength=45, ylength=45, timeout = 1000000):
    gates = gates.copy()
    values = {f"x{ii:02d}": bool((x//2**ii)%2) for ii in range(xlength)}
    values.update({f"y{ii:02d}": bool((y//2**ii)%2) for ii in range(ylength)})
    ii=0
    while gates:
        if ii > timeout: print("break",end=""); break
        current = gates.pop(0)
        if current[0] in values and current[1] in values:
            if current[2] == "AND":
                values[current[3]] = values[current[0]] and values[current[1]]
            elif current[2] == "OR":
                values[current[3]] = values[current[0]] or values[current[1]]
            elif current[2] == "XOR":
                values[current[3]] = values[current[0]] != values[current[1]]
        else: gates.append(current)
        ii += 1
    # z = sum(2**(n)*zvars[n] for n in range(len(zvars)))
    return [values[k] for k in sorted(list(values.keys())) if k[0]=="z"]

def check_circuit(gates:list, upto=45, printlittle=False):
    x_in, y_in = 'x00', 'y00'
    thexor = [gate[3] for gate in gates if {gate[0], gate[1]}=={x_in, y_in} and gate[2]=="XOR"][0]
    theand = [gate[3] for gate in gates if {gate[0], gate[1]}=={x_in, y_in} and gate[2]=="AND"][0]
    carry = theand
    print("x_in, y_in, thexor, theand, interm, out, carry")
    for n in range(1,upto+1):
        if printlittle: print("",end="\r")
        else: print("")
        x_in, y_in = f'x{n:02d}', f'y{n:02d}'
        try:
            theand = [gate[3] for gate in gates if {gate[0], gate[1]}=={x_in, y_in} and gate[2]=="AND"][0]
            thexor = [gate[3] for gate in gates if {gate[0], gate[1]}=={x_in, y_in} and gate[2]=="XOR"][0]
        except IndexError:
            print("\n", x_in, y_in, "in", [gate[3] for gate in gates if {gate[0], gate[1]}=={x_in, y_in}])
            break
        try:
            interm = [gate[3] for gate in gates if {gate[0], gate[1]}=={thexor, carry} and gate[2]=="AND"][0]
            out    = [gate[3] for gate in gates if {gate[0], gate[1]}=={thexor, carry} and gate[2]=="XOR"][0]
        except IndexError:
            print("\n", x_in, y_in, "intermed", [gate[3] for gate in gates if {gate[0], gate[1]}=={thexor, carry}])
            break
        try:
            carry  = [gate[3] for gate in gates if {gate[0], gate[1]}=={interm, theand} and gate[2]=="OR"][0]
        except IndexError:
            print("\n", x_in, y_in, "carry", [gate[3] for gate in gates if {gate[0], gate[1]}=={interm, theand}])
            break
        print(x_in, y_in, thexor, theand, interm, out, carry, end="")

    return x_in, y_in, thexor, theand, interm, out, carry

def swap_outs(v1, v2):
    v1_idx = [idx for idx,gate in enumerate(gates) if gate[3]==v1][0]
    v2_idx = [idx for idx,gate in enumerate(gates) if gate[3]==v2][0]
    gates[v1_idx] = (gates[v1_idx][0], gates[v1_idx][1], gates[v1_idx][2], v2)
    gates[v2_idx] = (gates[v2_idx][0], gates[v2_idx][1], gates[v2_idx][2], v1)

In [116]:
import random

#everything OK up to bit 14
for _ in range (1000):
    x, y = random.randint(0,2**14-1), random.randint(0,2**14-1)
    z_output = run_gates(gates, x, y, timeout=10000)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score != 46: print(x,y,x+y, sum(2**(n)*z_output[n] for n in range(46)) )

In [7]:
check_circuit(gates, printlittle=False)

x_in, y_in, thexor, theand, interm, out, carry

x01 y01 kgn hkv kmk z01 gcm
x02 y02 pfd jmb rgn z02 tmr
x03 y03 kpp npd fcc z03 hmn
x04 y04 jkv fnh ght z04 wjj
x05 y05 gpv vjt swv z05 thf
x06 y06 fsb wcw brr z06 twm
x07 y07 gsg fpb nfb z07 sgm
x08 y08 tsj jbk vsq z08 tkg
x09 y09 sjq drj pjh z09 vwd
x10 y10 vmf tgt nrg z10 rkk
x11 y11 gwv gnb srh z11 jvd
x12 y12 hgm fgv kpt z12 bkq
x13 y13 fpk vnf wmm z13 kds
x14 y14 shm qgm gnp z14 ccp
x15 y15 hhw mnh snp fph z15

 x16 y16 intermed []


('x16', 'y16', 'cbg', 'dcw', 'snp', 'fph', 'z15')

In [9]:
#z was assigned something other than out. Swap z and the one assigned to out
to_swap = {'fph', 'z15'}
swap_outs('fph', 'z15')
check_circuit(gates,printlittle=True)

x_in, y_in, thexor, theand, interm, out, carry
x20 y20 sbs frw pvq z20 nsp
 x21 y21 carry []


('x21', 'y21', 'tqh', 'z21', 'vqn', 'gds', 'nsp')

In [10]:
#again, z was assigned something other than out. Swap z and the one assigned to out
to_swap.update({'gds', 'z21'})
swap_outs('gds', 'z21')
check_circuit(gates, 30, printlittle=True)

x_in, y_in, thexor, theand, interm, out, carry
x29 y29 dcn tqc rjb z29 dwm
 x30 y30 intermed []


('x30', 'y30', 'jrs', 'wrk', 'rjb', 'z29', 'dwm')

In [11]:
#z was properly assigned to out. So check the new thexor and theand to see if they connect correctly to 1XOR+1AND and 1OR, resp.
[gate for gate in gates if 'jrs' in {gate[0], gate[1]}], [gate for gate in gates if 'wrk' in {gate[0], gate[1]}]

([('hjm', 'jrs', 'OR', 'hpt')],
 [('wrk', 'dwm', 'XOR', 'z30'), ('wrk', 'dwm', 'AND', 'hjm')])

In [12]:
#their connections are swapped. Swap them
to_swap.update({'jrs', 'wrk'})
swap_outs('jrs', 'wrk')
check_circuit(gates, 34, printlittle=True)

x_in, y_in, thexor, theand, interm, out, carry
x33 y33 grk qwt bwv z33 ksm
 x34 y34 carry []


('x34', 'y34', 'fcv', 'dfq', 'z34', 'cqk', 'ksm')

In [13]:
#once again, z was assigned something other than out. Swap z and the one assigned to out
to_swap.update({'cqk', 'z34'})
swap_outs('cqk', 'z34')
check_circuit(gates, 45, printlittle=True)

x_in, y_in, thexor, theand, interm, out, carry
x44 y44 smj wcr swb z44 z45
 x45 y45 in []


('x45', 'y45', 'smj', 'wcr', 'swb', 'z44', 'z45')

In [14]:
#only input 45 missing, because inputs go up to 44. We're done
",".join(sorted(to_swap))

'cqk,fph,gds,jrs,wrk,z15,z21,z34'

Trash (lots and lots of)

In [ ]:
def get_downstream(targets, gates):
    downstream_of = {}
    for init_var in [f"x{x:02d}" for x in targets]:# + [f"y{y:02d}" for y in faulty]:
        downstream_of[init_var] = set()
        nextup = set()
        for gate in gates:
            if gate[0] == init_var or gate[1] == init_var:
                nextup.add(gate[3])
                downstream_of[init_var].add(gate[3])
        while nextup:
            var = nextup.pop()
            for gate in gates:
                if gate[0] == var or gate[1] == var:
                    downstream_of[init_var].add(gate[3])
                    if gate[3][0]!="z": nextup.add(gate[3])
    return downstream_of

def get_upstream(targets, gates):
    upstream_of = {}
    for init_var in targets:#[zvar for eachset in faulty_inout.values() for zvar in eachset]:
        upstream_of[init_var] = set()
        nextup = set()
        for gate in gates:
            if gate[3] == init_var:
                nextup.update({gate[0], gate[1]})
                upstream_of[init_var].update({gate[0], gate[1]})
        while nextup:
            var = nextup.pop()
            for gate in gates:
                if gate[3] == var:
                    upstream_of[init_var].update({gate[0], gate[1]})
                    if gate[0][0] not in "xy": nextup.update({gate[0], gate[1]})
    return upstream_of

In [73]:
import random

#everything OK up to bit 30
for _ in range (1000):
    x, y = random.randint(0,2**29-1), random.randint(0,2**29-1)
    z_output = run_gates(gates, x, y, timeout=10000)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score != 46: print(x,y,x+y, sum(2**(n)*z_output[n] for n in range(46)) )

In [25]:
upstream_of = get_upstream([f"z{zvar:02d}" for zvar in range(16)], orig_gates)
# for key in upstream_of: print(key,":",sorted(list(upstream_of[key])))
for zvar in range(16):
    # if not upstream_of[f"z{zvar:02d}"].issubset(upstream_of[f"z{zvar+1:02d}"]): print(zvar)
    print(len(upstream_of[f"z{zvar:02d}"]))    

2
6
12
18
24
30
36
42
48
54
60
66
72
78
84
92


In [7]:
#one-hot x and y to find problems
pos_with_errors = set()
y=0
for xpos in range(45):
    x = 2**xpos
    z_output = run_gates(orig_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: pos_with_errors.add((xpos,-1))
x=0
for ypos in range(45):
    y = 2**ypos
    z_output = run_gates(orig_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: pos_with_errors.add((-1, ypos))
for xpos in range(45):
    if (xpos,-1) in pos_with_errors: continue
    for ypos in range(45):
        if (-1,ypos) in pos_with_errors: continue
        x, y = 2**xpos, 2**ypos
        z_output = run_gates(orig_gates, x, y)
        z_target = binarize(x+y, 46)
        score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
        if score!=46: pos_with_errors.add((xpos, ypos))
pos_with_errors

{(-1, 15),
 (-1, 21),
 (-1, 30),
 (-1, 34),
 (14, 14),
 (15, -1),
 (20, 20),
 (21, -1),
 (30, -1),
 (33, 33),
 (34, -1)}

In [8]:
faulty_inout = {}
y=0
for xpos in [15,21,30,34]:
    x = 2**xpos
    z_output = run_gates(orig_gates, x, y)
    z_target = binarize(x+y, 46)
    coincid = [z1==z2 for z1,z2 in zip(z_output, z_target)]
    score = sum(coincid)
    faulty_inout[xpos] = {f"z{pos:02d}" for pos in range(46) if not coincid[pos]}
    # print(xpos, score)
    # print([int(m) for m in coincid])
    # print([int(m) for m in z_output])
faulty_inout

{15: {'z15', 'z16'},
 21: {'z21', 'z22'},
 30: {'z30', 'z31'},
 34: {'z34', 'z35'}}

So we conclude the miswiring is somewhat behaved, doesn't connect wildly different things. But it does invert the output of $z_j$ and $z_{j+1}$ for the input $x_j$ (but not for input $x_{j+1}$)

In [9]:
#mapping on what faulty x,y variables each output z depends on... This should gives us a clue
faulty = {15,21,30,34} #{pair[1] for pair in pos_with_errors if pair[0]==-1}
def get_downstream(faulty):
    downstream_of = {}
    for init_var in [f"x{x:02d}" for x in faulty]:# + [f"y{y:02d}" for y in faulty]:
        downstream_of[init_var] = set()
        nextup = set()
        for gate in orig_gates:
            if gate[0] == init_var or gate[1] == init_var:
                nextup.add(gate[3])
                downstream_of[init_var].add(gate[3])
        while nextup:
            var = nextup.pop()
            for gate in orig_gates:
                if gate[0] == var or gate[1] == var:
                    downstream_of[init_var].add(gate[3])
                    if gate[3][0]!="z": nextup.add(gate[3])
    return downstream_of

downstream_of = get_downstream(faulty)
for key in downstream_of:
    print(key,":",sorted(list(downstream_of[key])))
# [downstream_of[f"x{x:02d}"] == downstream_of[f"y{x:02d}"] for x in faulty]

x34 : ['bbr', 'bbt', 'bjj', 'chn', 'chp', 'cqk', 'dfq', 'fcv', 'fwm', 'ggp', 'hdt', 'jbc', 'jvv', 'ncn', 'nmb', 'nrd', 'pjj', 'prp', 'ptm', 'qbr', 'swb', 'ttq', 'vpj', 'z34', 'z35', 'z36', 'z37', 'z38', 'z39', 'z40', 'z41', 'z42', 'z43', 'z44', 'z45']
x21 : ['bbr', 'bbt', 'bjj', 'btt', 'bwv', 'cfb', 'chn', 'chp', 'cqk', 'cqp', 'dwm', 'fpp', 'fsh', 'fvv', 'fwm', 'gds', 'ggp', 'gjd', 'gpr', 'hdt', 'hjm', 'hpt', 'jbc', 'jjs', 'jvv', 'khc', 'khs', 'ksm', 'mkc', 'ncn', 'nmb', 'nns', 'nrd', 'nsj', 'pct', 'pjj', 'prp', 'ptm', 'qbr', 'qkf', 'rjb', 'rwc', 'swb', 'tqh', 'tqk', 'ttq', 'vpj', 'vqn', 'wkh', 'z21', 'z22', 'z23', 'z24', 'z25', 'z26', 'z27', 'z28', 'z29', 'z30', 'z31', 'z32', 'z33', 'z34', 'z35', 'z36', 'z37', 'z38', 'z39', 'z40', 'z41', 'z42', 'z43', 'z44', 'z45']
x30 : ['bbr', 'bbt', 'bjj', 'bwv', 'chn', 'chp', 'cqk', 'fwm', 'ggp', 'gpr', 'hdt', 'hjm', 'hpt', 'jbc', 'jrs', 'jvv', 'khs', 'ksm', 'ncn', 'nmb', 'nrd', 'pct', 'pjj', 'prp', 'ptm', 'qbr', 'qkf', 'swb', 'ttq', 'vpj', 'wrk',

In [10]:
targets = [f"z{zvar+delta:2d}" for zvar in faulty for delta in [-1,0,1]]
def get_upstream(targets, gates):
    upstream_of = {}
    for init_var in targets:#[zvar for eachset in faulty_inout.values() for zvar in eachset]:
        upstream_of[init_var] = set()
        nextup = set()
        for gate in gates:
            if gate[3] == init_var:
                nextup.update({gate[0], gate[1]})
                upstream_of[init_var].update({gate[0], gate[1]})
        while nextup:
            var = nextup.pop()
            for gate in gates:
                if gate[3] == var:
                    upstream_of[init_var].update({gate[0], gate[1]})
                    if gate[0][0] not in "xy": nextup.update({gate[0], gate[1]})
    return upstream_of

upstream_of = get_upstream(targets, orig_gates)
for key in upstream_of:
    print(key,":",sorted(list(upstream_of[key])))

z33 : ['bkq', 'brr', 'bsb', 'btt', 'cbg', 'ccp', 'cfb', 'ckk', 'cqp', 'dcn', 'dcw', 'drj', 'dwm', 'fcc', 'fcw', 'fgv', 'fjh', 'fng', 'fnh', 'fpb', 'fph', 'fpk', 'fpp', 'frw', 'fsb', 'fsh', 'fvv', 'gcm', 'gds', 'ght', 'gjd', 'gmg', 'gnb', 'gnp', 'gpr', 'gpv', 'grk', 'gsg', 'gwv', 'hbq', 'hgm', 'hhw', 'hjm', 'hkv', 'hkw', 'hmn', 'hpt', 'hpw', 'hqj', 'hwf', 'jbk', 'jdw', 'jhj', 'jjs', 'jkc', 'jkv', 'jmb', 'jrs', 'jvd', 'kds', 'kgn', 'khc', 'khs', 'kmk', 'kpp', 'kpt', 'ksw', 'mch', 'mkc', 'msb', 'nbd', 'nfb', 'nkc', 'nns', 'npd', 'nrg', 'nsj', 'nsp', 'nsq', 'pct', 'pdf', 'pfd', 'pjh', 'pks', 'pmj', 'psh', 'pvq', 'qcf', 'qgm', 'qkf', 'rfw', 'rgn', 'rjb', 'rkk', 'rmj', 'rwc', 'sbs', 'sgm', 'sgs', 'shm', 'sjk', 'sjq', 'sms', 'srh', 'swv', 'tgt', 'thf', 'tkg', 'tmr', 'tqc', 'tqh', 'tqk', 'tsj', 'twm', 'vjt', 'vmf', 'vnf', 'vnn', 'vqn', 'vsk', 'vsq', 'vwd', 'wcw', 'wdg', 'wjj', 'wkh', 'wmm', 'wrk', 'x00', 'x01', 'x02', 'x03', 'x04', 'x05', 'x06', 'x07', 'x08', 'x09', 'x10', 'x11', 'x12', 'x13',

z21 clearly wrong

In [11]:
x,y = 2**21, 0
v1= 'z21'
v1_idx = [idx for idx,gate in enumerate(orig_gates) if gate[3]==v1][0]
to_swap = set()
for v2 in upstream_of['z22'].difference(upstream_of['z20']).difference({'x21','x22','y21','y22'}):
    v2_idx = [idx for idx,gate in enumerate(orig_gates) if gate[3]==v2][0]

    new_gates = orig_gates.copy()
    new_gates[v1_idx] = (new_gates[v1_idx][0], new_gates[v1_idx][1], new_gates[v1_idx][2], v2)
    new_gates[v2_idx] = (new_gates[v2_idx][0], new_gates[v2_idx][1], new_gates[v2_idx][2], v1)
    
    z_output = run_gates(new_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))

    if score==46:
        to_swap.update({v1,v2})
        print("worked", v1,v2)
        break
    
gates_1 = new_gates.copy()
pos_with_errors_1 = set()
y=0
for xpos in range(45):
    x = 2**xpos
    z_output = run_gates(new_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: pos_with_errors_1.add((xpos,-1))
x=0
for ypos in range(45):
    y = 2**ypos
    z_output = run_gates(new_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: pos_with_errors_1.add((-1, ypos))
for xpos in range(45):
    if (xpos,-1) in pos_with_errors_1: continue
    for ypos in range(45):
        if (-1,ypos) in pos_with_errors_1: continue
        x, y = 2**xpos, 2**ypos
        z_output = run_gates(new_gates, x, y)
        z_target = binarize(x+y, 46)
        score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
        if score!=46: pos_with_errors_1.add((xpos, ypos))
pos_with_errors_1

worked z21 gds


{(-1, 15),
 (-1, 30),
 (-1, 34),
 (14, 14),
 (15, -1),
 (30, -1),
 (33, 33),
 (34, -1)}

In [145]:
x,y = 2**15, 0
v1= 'z15'
v1_idx = [idx for idx,gate in enumerate(gates_1) if gate[3]==v1][0]
for v2 in upstream_of['z16'].difference(upstream_of['z14']).difference({'x15','x16','y15','y16'}):
    v2_idx = [idx for idx,gate in enumerate(gates_1) if gate[3]==v2][0]

    new_gates = gates_1.copy()
    new_gates[v1_idx] = (new_gates[v1_idx][0], new_gates[v1_idx][1], new_gates[v1_idx][2], v2)
    new_gates[v2_idx] = (new_gates[v2_idx][0], new_gates[v2_idx][1], new_gates[v2_idx][2], v1)
    
    z_output = run_gates(new_gates, x, y, timeout=10000)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))

    if score==46:
        to_swap.update({v1,v2})
        print("worked", v1,v2)
        break    

gates_2 = new_gates.copy()

pos_with_errors_2 = set()
y=0
for xpos in range(45):
    x = 2**xpos
    z_output = run_gates(new_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: pos_with_errors_2.add((xpos,-1))
x=0
for ypos in range(45):
    y = 2**ypos
    z_output = run_gates(new_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: pos_with_errors_2.add((-1, ypos))
for xpos in range(45):
    if (xpos,-1) in pos_with_errors_2: continue
    for ypos in range(45):
        if (-1,ypos) in pos_with_errors_2: continue
        x, y = 2**xpos, 2**ypos
        z_output = run_gates(new_gates, x, y)
        z_target = binarize(x+y, 46)
        score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
        if score!=46: pos_with_errors_2.add((xpos, ypos))
pos_with_errors_2

breakbreakbreakbreakworked z15 fph


{(-1, 30), (-1, 34), (30, -1), (33, 33), (34, -1)}

In [149]:
x,y = 2**30, 0
v1= 'z30'
v1_idx = [idx for idx,gate in enumerate(gates_2) if gate[3]==v1][0]
for v2 in upstream_of['z31'].difference(upstream_of['z29']).difference({'x30','x31','y30','y31'}):
    v2_idx = [idx for idx,gate in enumerate(gates_2) if gate[3]==v2][0]

    new_gates = gates_2.copy()
    new_gates[v1_idx] = (new_gates[v1_idx][0], new_gates[v1_idx][1], new_gates[v1_idx][2], v2)
    new_gates[v2_idx] = (new_gates[v2_idx][0], new_gates[v2_idx][1], new_gates[v2_idx][2], v1)
    
    min_score = 100
    for x,y in [(2**29,2**29),(2**30,0),(2**31,0)]:
        z_output = run_gates(new_gates, x, y, timeout=1000000)
        z_target = binarize(x+y, 46)
        min_score = min(sum(z1==z2 for z1,z2 in zip(z_output, z_target)), min_score)

    if min_score==46:
        to_swap.update({v1,v2})
        print("worked", v1,v2)
        break    

gates_3 = new_gates.copy()

pos_with_errors_3 = set()
y=0
for xpos in range(45):
    x = 2**xpos
    z_output = run_gates(new_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: pos_with_errors_3.add((xpos,-1))
x=0
for ypos in range(45):
    y = 2**ypos
    z_output = run_gates(new_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: pos_with_errors_3.add((-1, ypos))
for xpos in range(45):
    if (xpos,-1) in pos_with_errors_3: continue
    for ypos in range(45):
        if (-1,ypos) in pos_with_errors_3: continue
        x, y = 2**xpos, 2**ypos
        z_output = run_gates(new_gates, x, y)
        z_target = binarize(x+y, 46)
        score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
        if score!=46: pos_with_errors_3.add((xpos, ypos))
pos_with_errors_3

breakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreak

{(-1, 34), (29, 29), (33, 33), (34, -1)}

In [138]:
to_swap

{'fph', 'fsh', 'z15', 'z21'}

In [70]:
to_swap = set()
breakout = False
for v1 in downstream_of['x34']:
    if breakout: break
    v1_idx = [idx for idx,gate in enumerate(orig_gates) if gate[3]==v1][0]
    for v2 in downstream_of['x34'].difference({v1}):
        v2_idx = [idx for idx,gate in enumerate(orig_gates) if gate[3]==v2][0]

        new_gates = orig_gates.copy()
        new_gates[v1_idx] = (new_gates[v1_idx][0], new_gates[v1_idx][1], new_gates[v1_idx][2], v2)
        new_gates[v2_idx] = (new_gates[v2_idx][0], new_gates[v2_idx][1], new_gates[v2_idx][2], v1)

        x,y = 2**34, 0
        z_output = run_gates(new_gates, x, y)
        z_target = binarize(x+y, 46)
        score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))

        if score==46:
            to_swap.update({v1,v2})
            breakout = True
            break
if to_swap:
    x,y = 0, 2**34
    z_output = run_gates(new_gates, x, y)
    z_target = binarize(x+y, 46)
    score = sum(z1==z2 for z1,z2 in zip(z_output, z_target))
    if score!=46: print("x OK, but not y")
    else: print("x and y OK")
v1, v2

breakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreakbreak

KeyboardInterrupt: 

In [64]:
to_swap

set()

In [60]:
[idx for idx,gate in enumerate(orig_gates) if gate[3]==v1], orig_gates[181], v1

([181], ('kgq', 'ttq', 'OR', 'hdt'), 'hdt')

More trash

In [327]:
#the idea here is to try 1 swap at a time and see if the score improves. But I don't like this idea, actually...
def swap_gates(gates:list, i:int, j:int):
    gates = gates.copy()
    temp = gates[i][-1]
    gates[i] = (gates[i][0], gates[i][1], gates[i][2], gates[j][-1])
    gates[j] = (gates[j][0], gates[j][1], gates[j][2], temp)
    del temp
    return gates

In [328]:
import random

score = 0
while score < len_zvars*.5:
    values = orig_values.copy()
    i,j = random.randint(0,len(orig_gates)-1), random.randint(0,len(orig_gates)-1)
    gates = swap_gates(orig_gates, i,j)
    zvars = run_gates(gates, values)
    score = sum(z==target for z,target in zip(zvars, binary_target))

print( i, j, score, len_zvars, end=", ")
for _ in range(10):
    x, y = random.randint(0,2**len(xvars)-1), random.randint(0,2**len(yvars)-1)
    binary_target = [bool(((x+y)//2**j)%2) for j in range(len_zvars)]
    values = {f"x{ii:02d}": bool((x//2**ii)%2) for ii in range(len(xvars))}
    values.update({f"y{ii:02d}": bool((y//2**ii)%2) for ii in range(len(yvars))})

    zvars = run_gates(gates, values)
    score = sum(z==target for z,target in zip(zvars, binary_target))
    print(f"{score/len_zvars:.2f}", end="p")

204 92 23 46, 0.80p0.87p0.87p0.72p0.76p0.87p0.74p0.80p0.89p0.83p

In [338]:
score = 0
while score < len_zvars*.55:
    values = orig_values.copy()
    gates = swap_gates(orig_gates, 204, 92)
    i,j = random.randint(0,len(orig_gates)-1), random.randint(0,len(orig_gates)-1)
    gates = swap_gates(gates, i,j)
    zvars = run_gates(gates, values)
    score = sum(z==target for z,target in zip(zvars, binary_target))

print( i, j, score, len_zvars, end=", ")
for _ in range(10):
    x, y = random.randint(0,2**len(xvars)-1), random.randint(0,2**len(yvars)-1)
    binary_target = [bool(((x+y)//2**j)%2) for j in range(len_zvars)]
    values = {f"x{ii:02d}": bool((x//2**ii)%2) for ii in range(len(xvars))}
    values.update({f"y{ii:02d}": bool((y//2**ii)%2) for ii in range(len(yvars))})

    zvars = run_gates(gates, values)
    score = sum(z==target for z,target in zip(zvars, binary_target))
    print(f"{score/len_zvars:.2f}", end="p")

149 146 27 46, 0.48p0.57p0.54p0.43p0.48p0.54p0.70p0.59p0.57p0.61p

In [207]:
 46 42 28 46, 0.85 0.89 0.74 0.74 0.91 0.87 0.72 0.93 0.87 0.61

([('y24', 'x24', 'XOR', 'jhj'),
  ('y00', 'x00', 'AND', 'ksw'),
  ('rmj', 'khc', 'OR', 'cqp'),
  ('y22', 'x22', 'AND', 'pdf'),
  ('fsb', 'thf', 'AND', 'brr')],
 222)

In [47]:
z_condensed = [values[k] for k in sorted(list(values.keys())) if k[0]=="z"]
sum(2**(n)*z_condensed[n] for n in range(len(z_condensed)))

0